In [1]:
import wikipedia
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import re
from urllib.parse import unquote
import wikipediaapi

In [119]:
class extract_portal_wikipedia:
    
    def __init__(self, url_portal):
        self.url_portal = url_portal

    def extract_url(self):

        req = requests.get(self.url_portal)
        soup = BeautifulSoup(req.text, "lxml")

        tmp_links = []
        urls = []

        for i in soup.find_all(name='li'):
            for link in i.find_all('a', href=True):
                tmp_links.append(link['href'])
        for i in tmp_links:
            article = re.findall('/wiki/(.*?)$', i)
            if article != []:
                urls.append(unquote(article[0]))
        return urls

    def extract_resumes(self, urls):
        resumes = []
        articles = [] 
        titres = [] 
        wiki_wiki = wikipediaapi.Wikipedia(language='fr', extract_format=wikipediaapi.ExtractFormat.WIKI) 
        for url in urls:
            print(f'je test {url}')
            resume = wiki_wiki.page(url)
            if resume.exists():
                print(f'{url} existe')
                resumes.append(resume.summary)
                if resume.text!='' and resume.text != None:
                    article = resume.text
                    #article = re.search("\\n\\n", article).start()
                    article = re.sub('(\\n\\n).*(\\n)', "", article)
                    article = re.sub("\\n", "", article)
                    articles.append(article)
                else:
                    articles.append(resume.text) 
                titres.append(resume.title)
            else:
                print(f"{url} n'existe pas, on essaie donc {url.replace('_', ' ')}")
                resume = wiki_wiki.page(url.replace('_', ' '))

                if not resume.exists():
                    print(f"{url}  & {url.replace('_', ' ')} n'existe pas")
                    resume = None
                else:
                    print(1)
                    resumes.append(resume.summary)
                    if resume.text!='':
                        article = re.sub('(\\n\\n).*(\\n)', "", article)
                        article = re.sub("\\n", "", article)
                        articles.append(article)
                    else:
                        articles.append(resume.text) 
                    titres.append(resume.title)
            del(url)

        return  titres, resumes, articles



In [120]:
portail_economie = extract_portal_wikipedia('https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Portail:%C3%89conomie/Articles_li%C3%A9s&from=B')
url_economie = portail_economie.extract_url()
resumes = portail_economie.extract_resumes(url_economie[0:20])

import pandas as pnd
df = pnd.DataFrame(resumes)

je test Catégorie:Économie
Catégorie:Économie existe
je test Portail:Économie
Portail:Économie existe
je test Projet:Économie
Projet:Économie existe
je test Discussion_Projet:Économie
Discussion_Projet:Économie existe
je test Catégorie:Portail:Économie/Articles_liés
Catégorie:Portail:Économie/Articles_liés existe
je test Spécial:ArbreCatégorie/Portail:Économie/Articles_liés
Spécial:ArbreCatégorie/Portail:Économie/Articles_liés n'existe pas, on essaie donc Spécial:ArbreCatégorie/Portail:Économie/Articles liés
Spécial:ArbreCatégorie/Portail:Économie/Articles_liés  & Spécial:ArbreCatégorie/Portail:Économie/Articles liés n'existe pas
je test Spécial:Recherche/incategory:"Portail:Économie/Articles_liés"
Spécial:Recherche/incategory:"Portail:Économie/Articles_liés" n'existe pas, on essaie donc Spécial:Recherche/incategory:"Portail:Économie/Articles liés"
Spécial:Recherche/incategory:"Portail:Économie/Articles_liés"  & Spécial:Recherche/incategory:"Portail:Économie/Articles liés" n'existe pas

In [121]:
df.iloc[1,5]

"Le B-Park est un centre commercial situé dans la ville belge de Bruges, au nord du parc d'affaires de la Blauwe Toren dans le quartier de Saint-Pierre-sur-la-Digue. Inauguré le 22 octobre 2008, il comprend aujourd'hui, en mai 2014, 16 enseignes sur une superficie de 42 000 m2.\nSon parc de stationnement peut accueillir environ 2000 voitures et il y a également un stationnement à vélos couvert pour 400 vélos."

In [122]:
df.iloc[2,5]

"Le B-Park est un centre commercial situé dans la ville belge de Bruges, au nord du parc d'affaires de la Blauwe Toren dans le quartier de Saint-Pierre-sur-la-Digue. Inauguré le 22 octobre 2008, il comprend aujourd'hui, en mai 2014, 16 enseignes sur une superficie de 42 000 m2.Son parc de stationnement peut accueillir environ 2000 voitures et il y a également un stationnement à vélos couvert pour 400 vélos.Le retail park a été développé par Codic.En août 2007, la construction du centre commercial « B-Park » a débuté, l'ouverture du centre commercial au public a ensuite eu lieu le mercredi 22 octobre 2008.Le centre commercial du B-Park abrite le plus grand hypermarché de la Région flamande, il s'agit d'un Carrefour Planet à 14 000 m2, les autres magasins les plus grands sont Décathlon (4 600 m2), Krëfel (3 000 m2), DreamLand (3 000 m2) et Leen Bakker (2 000 m2). Les autres enseignes présentes sont Auto 5, AVA, Baby 2000, Biosleepworld, Èggo, Jakobsen Wonen, Kwantum, Lunch Garden, Maison

In [147]:
df_transpose = df.T
df_transpose.columns = ['Titre', 'Resume', 'Texte']
df_transpose

,Titre,Resume,Texte
0,Catégorie:Économie,,
1,Portail:Économie,,
2,Projet:Économie,,
3,Discussion Projet:Économie,"== Un diff à vérifier ==\nBonjour, quelqu'un p...","== Un diff à vérifier ==Bonjour, quelqu'un pou..."
4,Catégorie:Portail:Économie/Articles liés,,
5,B-Park,Le B-Park est un centre commercial situé dans ...,Le B-Park est un centre commercial situé dans ...
6,B. Braun,"Le groupe allemand B. Braun, fondé en 1839, es...","Le groupe allemand B. Braun, fondé en 1839, es..."
7,B/E Aerospace,B/E Aerospace est un équipementier américain p...,B/E Aerospace est un équipementier américain p...
8,B&B Hotels,"B&B Hotels est une chaîne d'hôtels, créée en 1...","B&B Hotels est une chaîne d'hôtels, créée en 1..."
9,B&H Airlines,B&H Airlines était une compagnie aérienne basé...,B&H Airlines était une compagnie aérienne basé...


In [148]:
df_transpose['summary_lenght']=df_transpose['Resume'].apply(len)
df_transpose

,Titre,Resume,Texte,summary_lenght
0,Catégorie:Économie,,,0
1,Portail:Économie,,,0
2,Projet:Économie,,,0
3,Discussion Projet:Économie,"== Un diff à vérifier ==\nBonjour, quelqu'un p...","== Un diff à vérifier ==Bonjour, quelqu'un pou...",430
4,Catégorie:Portail:Économie/Articles liés,,,0
5,B-Park,Le B-Park est un centre commercial situé dans ...,Le B-Park est un centre commercial situé dans ...,410
6,B. Braun,"Le groupe allemand B. Braun, fondé en 1839, es...","Le groupe allemand B. Braun, fondé en 1839, es...",559
7,B/E Aerospace,B/E Aerospace est un équipementier américain p...,B/E Aerospace est un équipementier américain p...,140
8,B&B Hotels,"B&B Hotels est une chaîne d'hôtels, créée en 1...","B&B Hotels est une chaîne d'hôtels, créée en 1...",271
9,B&H Airlines,B&H Airlines était une compagnie aérienne basé...,B&H Airlines était une compagnie aérienne basé...,653


In [157]:
df_transpose['texte_sans_resume']=df_transpose['Texte'].str.slice(start = df_transpose['summary_lenght'])

In [156]:
type(df_transpose['summary_lenght'])

pandas.core.series.Series